# Data Wrangling with Pandas

We've seen how to get data with Python. Now let's do some stuff! From here on, we're going to mostly use the PyData stack rather than Python built-in functionality.

Our objective in this section is to learn enough to clean the larger sample of Chicago Health Inspection data and get it ready for modeling.

## Preliminaries: DataFrames

As mentioned, the core data structure in pandas is called a DataFrame. A DataFrame is a tabular data structure, holding many columns, similar to a spreadsheet.

The **Key Features** are

* Easy handling of **missing data**
* **Size mutability**: columns can be inserted and deleted from DataFrames
* Automatic and explicit **data alignment**: objects can be explicitly aligned to a set of labels, or the data can be aligned automatically
* Powerful, flexible **group by functionality** to perform split-apply-combine operations on data sets
* Intelligent label-based **slicing**, **fancy indexing**, and **subsetting** of large data sets
* Intuitive **merging and joining** data sets
* Flexible **reshaping and pivoting** of data sets
* **Hierarchical labeling** of axes
* Robust **IO tools** for loading data from flat files, Excel files, databases, and HDF5
* **Time series functionality**: 
  * date range generation and frequency conversion
  * moving window statistics
  * moving window linear regressions
  * date shifting and lagging, etc.

In [ ]:
dta = pd.read_csv("data/health_inspection_chi.csv")

Pandas provides labelled **indices** to access rows and columns, should they have natural labels.

In [ ]:
dta.index

In [ ]:
dta.columns

For example, with this data set we have a natural unique identifier in the `inspection_id` column. We might wish to make this our index.

In [ ]:
dta.head()

In [ ]:
dta = dta.set_index('inspection_id')

In [ ]:
dta.index

In [ ]:
dta.head()

## Indexing

To look at a column from a DataFrame, you can either use attribute lookup.

In [ ]:
dta.address

Or you can use the **getitem** syntax that relies on square brackets `[]`, which is familiar from dealing with dictionaries (uses `__getitem__`).

In [ ]:
dta['address']

These two operations return pandas **Series** objects. **Series** are like single-column DataFrames. If you want to preserve the DataFrame type, index the DataFrame with a list.

In [ ]:
dta[['address']]

You can use this syntax to pull out multiple columns.

In [ ]:
dta[['address', 'inspection_date']]

You can index the rows, by using the **loc** and **iloc** accessors.

`loc` does *label-based* indexing.

In [ ]:
dta.loc[[1965287, 1329698]]

`iloc` on the other hand provides *integer-based* indexing. We can pass a list of rows integers.

In [ ]:
dta.iloc[[0, 2]]

Both support the Python **slice notation** (`start:stop:step`). This can be really powerful.

In [ ]:
dta.iloc[:5]

In [ ]:
dta.loc[:1335320]

Note that these inspection ids are *not* sorted, yet we can still use slice notation.

Of course, we can also combine row and index labeling.

In [ ]:
dta.iloc[:5, [0, 5]]

In [ ]:
dta.loc[:68091, ["address", "inspection_date"]]

## Cleaning Data for Types

So far, we've explicitly made an index. We may next want to convert to the dates to datetime types. Here we'll use the **apply** function to apply a function to each row of a Series.

In [ ]:
dta.inspection_date = dta.inspection_date.apply(pd.to_datetime)

In [ ]:
dta.inspection_date

Now let's cast zip code from a float to a string. Some zip codes can start with 0 (not in Chicago), and we need to account for that.

In [ ]:
import numpy as np


def float_to_zip(zip_code):
    if np.isnan(zip_code):
        return np.nan
    
    # 0 makes sure to left-pad with zero
    # zip codes have 5 digits
    # .0 means, we don't want anything after the decimal
    # f is for float
    zip_code = "{:05.0f}".format(zip_code)
    return zip_code

Here we use Python's **string formatting** facilities to convert from a numeric type to a string. Some of the zip codes are empty strings in the file. Pandas uses numpy's `NaN` to indicate missingness, so we'll return it here.

In [ ]:
dta.zip = dta.zip.apply(float_to_zip)

In [ ]:
dta.head()

DataFrames have a `dtypes` attribute for checking the data types. Pandas relies on NumPy's dtypes objects. Here we see that the `object` dtype is used to hold strings. This for technical reasons.

In [ ]:
dta.dtypes[['inspection_date', 'zip']]

We can also convert variables' types, using `astype`. Here, we'll explicitly cast to pandas Categorical type, which is the only non-native numpy type.

In [ ]:
dta.info()

In [ ]:
dta.results = dta.results.astype('category')
dta.risk = dta.risk.astype('category')
dta.inspection_type = dta.inspection_type.astype('category')
dta.facility_type = dta.facility_type.astype('category')

If we only select the categorical types, we can see some categorical variables descriptions.

We can use the `select_dtypes` method to pull out a DataFrame with only the asked for types.

In [ ]:
dta.select_dtypes(['category'])

Finally, we might want to exclude a column like `location` since we have the separate `latitude` and `longitude` columns. We can delete columns in a DataFrame using Python's built-in `del` statement.

In [ ]:
del dta['location']

## Dealing with Types using csv Reader

We can do everything that we did above by providing options to `pd.read_csv`.

We saw before that `csv` reads everything in as strings, `json` does some type conversion with facility for doing more, and `pandas` does a bit more type conversion but it isn't always what we want. For example, we want the zip codes to stay strings.

Let's take a look at how to do with pandas `read_csv`. First, we can use the `parse_dates` argument to read in the larger inspections data sample and tell pandas that one of our columns is a date column. We'll also go ahead and make `inspection_id` the index.

In [ ]:
dta = pd.read_csv(
    "data/health_inspection_chi.csv", 
    index_col="inspection_id",
    parse_dates=["inspection_date"]
)

Next, we want to turn the zip codes into strings. Here, we need to assume that the input (from the file) is a string as opposed to the above.

In [ ]:
import numpy as np


def float_to_zip(zip_code):
    # convert from the string in the file to a float
    try:
        zip_code = float(zip_code)
    except ValueError:  # some of them are empty
        return np.nan
    
    # 0 makes sure to left-pad with zero
    # zip codes have 5 digits
    # .0 means, we don't want anything after the decimal
    # f is for float
    zip_code = "{:05.0f}".format(zip_code)
    return zip_code

In [ ]:
float_to_zip('1234')

In [ ]:
float_to_zip('123456')

As another example of defensive programming, we have to make sure that empty strings are handled.

In [ ]:
float_to_zip('')

We can supply this function to the `converters` argument.

In [ ]:
dta = pd.read_csv(
    "data/health_inspection_chi.csv",
    converters={
        'zip': float_to_zip
    },
)

In [ ]:
dta.head()

To exclude location, we can take advantage of the fact that the `usecols` argument accepts a function to exclude `location`.

In [ ]:
dta = pd.read_csv(
    "data/health_inspection_chi.csv",
    usecols=lambda col: col != 'location'
)

Here we are using a **lambda function** that returns `False` for the location parameter. Lambda functions are what are known as anonymous functions, because they don't have a name. This kind of thing is precisely their intended use.

Here we use a function `lambda x: x` to map the identity function over a list.

In [ ]:
list(map(lambda x: x, [1, 2, 3]))

Finally, in a few cases we may want to take advantage of the pandas native `categorical` type. We can use the `dtype` argument for this, passing a dictionary of type mappings.

In [ ]:
dta = pd.read_csv(
    "data/health_inspection_chi.csv",
    dtype={
        'results': 'category',
        'risk': 'category',
        'inspection_type': 'category',
        'facility_type': 'category'
    }
)

In [ ]:
dta.risk.head()

## Exercise

Put all of the above `read_csv` options together in a single call to `read_csv`.

In [ ]:
# Type your solution here

In [ ]:
%load solutions/pandas_read_csv.py

## String Cleaning

Ok, let's start to dig into the data a little bit more. One of the things we're going to be really interested in exploring is the free text of the violations field.

The first thing to notice is that the violations field has null values in it.

In [ ]:
dta.info()

We may want to ask ourselves if these values are missing at random or if there is some reason there's no written violation field.

In [ ]:
dta.loc[dta.violations.isnull()].head()

It looks like we're ok. The next thing to notice is that the violation field actually has a lot of violations in the same field for the same visit.

In [ ]:
with pd.option_context("display.max_colwidth", 500):
    print(dta.violations.head())

Let's split these out to make a longer DataFrame where each violation is a single row. Pandas provides a nice way to munge string data through the `str` accessor on string columns.

```python
dta.violations.str.<TAB>
```

## Exercise

Let's see how many violations we have per visit. What does the distribution of violations look like? Explore the methods on the `str` accessor and, perhaps, the `quantile` method.

In [ ]:
# Type your solution here

In [ ]:
%load solutions/violation_distribution.py

Ok, we have a manageable number of violations. Let's split the violations and then turn them into a long DataFrame with a single row for each violation within each visit.

In [ ]:
violations = dta.violations.str.split("\|", expand=True)
violations.head()

When we `unstack` the DataFrame, we're left with what's called a `MultiIndex`. This index has two *levels* now. One is the original `inspection_id`. The other is the, rather meaningless, column names.

In [ ]:
violations.unstack().head()

Let's get rid of the empty rows first.

In [ ]:
violations = violations.unstack().dropna()

Now we can drop the column name level, which we don't need.

In [ ]:
violations.reset_index(level=0, drop=True, inplace=True)

In [ ]:
violations.head()

One last cleaning step may be helpful here. When we split on the pipe ('`|`'), we likely kept some surrounding whitespace. We can remove that.

In [ ]:
violations.str.startswith(" ").any()

In [ ]:
violations.str.strip().head()

In [ ]:
violations = violations.str.strip()

In [ ]:
((violations.str.startswith(" ").any()) | 
 (violations.str.endswith(" ").any()))

Later, we'll see how to combine these violations back with our original data to do some analysis.

## Working with Dates and Categoricals

Above, we used the `str` accessor on the DataFrame. This isn't the only convenient accessor that pandas provides. There is also the `dt` accessor for datetime types and the `cat` accessor for categorical types.

```python
dta.inspection_date.dt.<TAB>
```

In [ ]:
dta.inspection_date.head()

In [ ]:
dta.inspection_date.dt.month.head()

Now, let's take a look at the categorical types.

```python
dta.risk.cat.<TAB>
```

In [ ]:
dta.risk.head()

In [ ]:
dta.risk.cat.codes.head()